In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from proofreader.utils.vis import plot_3d
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from proofreader.data.cremi import prepare_cremi_vols

train_vols, test_vols = prepare_cremi_vols('../../dataset/cremi')

In [ ]:
from proofreader.data.splitter import SplitterDataset
from proofreader.data.augment import Augmentor
num_slices = [4, 4]
radius = 96
context_slices = 4
num_points = 1024

augmentor = Augmentor(center=True, shuffle=True, normalize=[125, 1250, 1250])
train_dataset = SplitterDataset(test_vols, num_slices, radius, context_slices, num_points=num_points, torch=True, open_vol=True, verbose=False, Augmentor=augmentor)
print(len(train_dataset))

In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
dataloader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
model, _ ,_ = build_full_model_from_config(config.model, config.dataset)
model = nn.DataParallel(model)
model = load_model(model, '../../330.ckpt', map_location=torch.device('cpu'))



In [ ]:
from proofreader.model.classifier import *
with torch.no_grad():
    count, acc = 0,0
    for step, batch in enumerate(dataloader):
        count += 1
        # get batch
        x, y = batch
        y_hat = model(x)
        pred = predict_class(y_hat)
        accs = get_accuracy(y, pred)
        print(accs)
        acc += accs['total_acc']
        print(round(acc/count, 3))


In [ ]:
from proofreader.utils.vis import plot_3d
%matplotlib widget
num = 10
lim = (-.1,.1)
lim = (lim,lim,lim)
print(lim)
for i in range(0, num):
    rand_i = random.randint(0, len(dataset))
    pc, label = dataset[rand_i]
    pc = np.swapaxes(pc, 0, 1)
    plot_3d(pc, lims=lim)



In [179]:
from proofreader.data.splitter import SplitterTest
from proofreader.data.augment import Augmentor
from proofreader.utils.torch import load_model
from proofreader.model.config import *
from torch.utils.data import DataLoader
import torch.nn as nn

config = get_config('cn_context_4_aug_small')

model, _ ,_ = build_full_model_from_config(config.model, config.dataset)
model = nn.DataParallel(model)
model = load_model(model, '../../330.ckpt', map_location=torch.device('cpu'))

num_slices = [1, 4]
radius = 96
context_slices = 4
num_points = 1024
augmentor = Augmentor(center=True, shuffle=True, normalize=[125, 1250, 1250])
tester = SplitterTest(test_vols, num_slices, radius, context_slices, num_points=num_points, Augmentor=augmentor)


In [205]:
tester.test_iter(test_vols[1], model, (6,10))



total neurites 206
step 1: acc 0.00
step 2: acc 0.50
step 3: acc 0.67
step 6: acc 0.75
step 7: acc 0.80
step 8: acc 0.83
step 9: acc 0.71
step 10: acc 0.75
step 11: acc 0.78
step 12: acc 0.80
step 13: acc 0.82
step 14: acc 0.75
step 16: acc 0.77
step 18: acc 0.79
step 19: acc 0.80
step 21: acc 0.75
step 24: acc 0.71
step 26: acc 0.67
step 27: acc 0.68
step 28: acc 0.65
step 29: acc 0.62
step 31: acc 0.64
step 32: acc 0.61
step 33: acc 0.62
step 35: acc 0.64
step 36: acc 0.65
step 37: acc 0.67
step 38: acc 0.64
step 39: acc 0.66
step 40: acc 0.67
step 43: acc 0.65
step 44: acc 0.66
step 45: acc 0.64
step 47: acc 0.65
step 49: acc 0.66
step 51: acc 0.67
step 52: acc 0.65


KeyboardInterrupt: 